# How to Create Subscriptions

> Set up recurring payments for services

This guide shows you how to create recurring subscription payments using FastStripe.

## Problem

You want to charge customers on a recurring basis (monthly, yearly, etc.) for a service or product.

## Solution

Use FastStripe's `subscription()` method which creates the necessary recurring price and checkout session.

In [ ]:
from faststripe.core import StripeApi
import os

sapi = StripeApi(os.environ['STRIPE_SECRET_KEY'])

## Monthly subscription

In [ ]:
# Create a monthly subscription
subscription = sapi.subscription(
    product_name='Pro Plan',
    amount_cents=1999,  # $19.99/month
    success_url='https://yoursite.com/welcome',
    cancel_url='https://yoursite.com/pricing',
    interval='month'
)

print(f"Subscription URL: {subscription.url}")

## Annual subscription with discount

In [ ]:
# Create an annual subscription (typically discounted)
annual_sub = sapi.subscription(
    product_name='Pro Plan',
    amount_cents=19999,  # $199.99/year (save $39.89)
    success_url='https://yoursite.com/welcome',
    cancel_url='https://yoursite.com/pricing',
    interval='year'
)

print(f"Annual subscription URL: {annual_sub.url}")

## Pre-fill customer email

In [ ]:
# Pre-fill customer email in checkout
subscription = sapi.subscription(
    product_name='Pro Plan',
    amount_cents=1999,
    success_url='https://yoursite.com/welcome',
    cancel_url='https://yoursite.com/pricing',
    customer_email='user@example.com'
)

print(f"Subscription with email: {subscription.url}")

## Manual subscription setup

For more control over subscription features:

In [ ]:
# Create product
product = sapi.products.create(
    name='Enterprise Plan',
    description='Full access to all features'
)

# Create recurring price
price = sapi.prices.create(
    product=product.id,
    unit_amount=9999,  # $99.99
    currency='usd',
    recurring={
        'interval': 'month',
        'usage_type': 'licensed'  # per-seat billing
    }
)

# Create subscription checkout
checkout = sapi.checkout_sessions.create(
    mode='subscription',
    line_items=[{
        'price': price.id,
        'quantity': 1
    }],
    success_url='https://yoursite.com/success?session_id={CHECKOUT_SESSION_ID}',
    cancel_url='https://yoursite.com/cancel',
    allow_promotion_codes=True,  # Enable promo codes
    subscription_data={
        'trial_period_days': 14  # 14-day free trial
    }
)

print(f"Enterprise subscription: {checkout.url}")

## Managing existing subscriptions

Retrieve and modify existing subscriptions:

In [ ]:
# List all subscriptions
subscriptions = sapi.subscriptions.fetch(limit=10)
print(f"Found {len(subscriptions.data)} subscriptions")

# Get a specific subscription
if subscriptions.data:
    sub = subscriptions.data[0]
    print(f"Subscription {sub.id}: {sub.status}")
    
    # Cancel subscription
    # cancelled_sub = sapi.subscriptions.update(sub.id, cancel_at_period_end=True)
    # print(f"Subscription will cancel at period end: {cancelled_sub.cancel_at_period_end}")

## Best practices

- Always use webhooks to handle subscription lifecycle events
- Consider offering free trials to increase conversions
- Use promotion codes for discounts and marketing campaigns
- Set up proper success/cancel URLs that match your user flow
- Test subscription flows thoroughly in test mode